# Goal of the project

We want to identify users that are most likely to convert to paying customers. We would like to build a scoring mechanism to rate different users. A customer with a higher rating ought to have a high chance of conversion,
while lower rated customers would be less likely to convert. We have a dataset containing historical
information about customers along with a binary label that shows whether they converted or not. The goal is to use this data to score potential new paid customers.

## Goal of this document

To complete an initial analysis of the available data and perform an evaluation in order to suggest a machine learning solution for this project. Part of the python code used for this notebook will be added to a another python file so that we can run it as a separate python script.

# Assumptions

The sensitive data has been already anonymized so there are no further actions from our side.

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Potential approach

The dataset is labeled so we could initially consider this project as a *supervised machine learning classification* problem. Thus, the idea would be to build a **prediction model** where we can tell if a lead is going to become a paid customer or not. The feasibility of such a model is totally dependent on the available data and how relevant is that data for the final classification of the lead.
Let´s start with the exploration and analysis of the provided dataset in order to detect any patterns or insights that might be helpful before building any model.

# EDA Steps
1. Exploration: to understand better the dataset.
2. Dataset cleaning: to remove not relevant data for the classifier.
    - Missing Data
    - Outliers
3. Data transformation: all features must be numerical. Check ranges.
    - Categorical Data
    - Scaling numerical data
4. Correlation analysis: to check dependencies between features and remove redundant data.
5. Saving the final dataset

# Exploration

We will perform some basic exploration about the different features, outliers, categorical data, missing data, balanced information, etc.

In [ ]:
df = pd.read_csv('../ub_dataset.csv')
df.head()

The first column of the dataset is not documented in the provided file, but after asking the origin we received the information that is an ordering index so we can delete it from our dataset since it is not relevant.

In [ ]:
df = df.iloc[:,1:]
df.head()

In [ ]:
df.info()

We have some string fields, so we will perform a separate analysis on categorical data to check how many different unique values do we have per each one.
The *signup_date* can give us an idea of the time range of the dataset. This is important if we are trying to find patterns in customers. It could happen that the patterns customers showed 5 years ago, are not valid anymore. Let´s perform some time analysis of the customers.

In [ ]:
min(df.signup_date)

In [ ]:
max(df.signup_date)

Thus the timeline of data is from 2008 to 2019, more than 10 years.

### Number of customers per year
We could analyze which years were most successful in converting customers according to the provided data

In [ ]:
df["signup_date"] = pd.to_datetime(df["signup_date"])
df["year"] = df["signup_date"].dt.year

In [ ]:
leads_per_year = df.groupby(["year", "label"]).size().to_frame(name='count').reset_index()
leads_per_year

In [ ]:
sns.scatterplot(data=leads_per_year, x="year", y="count", hue="label", palette="Set2")

There are no successful leads in data before 2014. This could be used to filter just recent data where we see that the volume increased and also conversions happened. Besides due to the imbalance between the volumes of each type one potential technique that could be used to compensate is downsampling, i.e. reducing the samples of the major class.

### Number of leads per month of the year

In [ ]:
df["month"] = df["signup_date"].dt.month
leads_per_month = df.groupby(["month", "label"]).size().to_frame(name='count').reset_index()
sns.scatterplot(data=leads_per_month, x="month", y="count", hue="label", palette="Set2")

In [ ]:
sns.scatterplot(data=leads_per_month[leads_per_month["label"]==1], x="month", y="count")

The summer months and specially September seem to be a good period of the year when more conversions happened. Winter months are the worst.

From the numerical features we can analyze the distributions.

In [ ]:
df.describe()

There are negative values for four fields. For the timezone this might be desired but for the other ones, the -1 value might mean "Null" or there is no information about this field.

# Data cleaning
We will use year to remove the first period with a very low volume of data that could introduce noise.

In [ ]:
len(df)

In [ ]:
df = df[df["year"]>2013]

In [ ]:
len(df)

### Features that cannot be used to predict

The *fakeId* is an internal identifier for the customer, so it is unique number and it will be a new number for a new lead. It is impossible to make any prediction just using this information so we can remove it from the list of relevant features. 
The *signup-date* is the timestamp of the signing moment so it gives us an idea of when the lead registered in the system. We completed before some initial analysis per year and per month. The year was used to filter relevant data and downsampling a bit the major class. We would like to keep month of the year and it might show some patterns. 
There is place for optimizations regarding this initial/basic *seasonality analysis* to probe some correlation between successful campaigns and the period of the year where the campaign was launched. This guess would require some extra time and conversations with marketing teams. Thus we discard signup_date and year. 

In [ ]:
df.drop(['fakeId', 'signup_date', 'year'], axis=1, inplace=True)

### Outliers 

The feature *place_within_tenant* is showing a maximal value of 261 very far away from the rest of the values. This could be an error or not, so it is a clear outlier but we do not have enough confidence in removing it. Besides the feature *company_employes_qty* is also showing some outliers with 4 orders of magnitude away from the 75% percentile. We decided not to remove any outliers but we consider it is worthy to mention them and to investigate them just to detect possible errors on the dataset.

### Missing Data

During the first exploration we already found some missing values in the form of negative values, -1, for some columns. We would like to transform these values into *nan* values to get an idea of which columns do have a huge volume of missing information.

In [ ]:
columns_fix = ['continent', 'country_code', 'city', 'region_code']
for col in columns_fix:
    df[col].replace(-1, np.nan, inplace = True)
df.describe()

In [ ]:
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})

In [ ]:
len(df.columns)

In [ ]:
# we have 17 columns
missing_value_df.sort_values('percent_missing', ascending=False, inplace=True)
missing_value_df.head(17)

After checking these percentages, we have three candidates with more than 50% of missing data, so these features are not going to bring much value as predictors for a prediction model. Thus we delete the top three in this list. 
An alternative approach, instead of deletion, could be to transform them into a *synthetic* boolean feature meaning True if we have info and False if we do not have information. Another approach is imputing the nan values with another new value, however, given the high percentage of missing data, we will be automatically creating a new dominant class.

In [ ]:
df.drop(['company_industrygroups', 'category_sectors', 'company_employes_qty'], axis=1, inplace=True)

For the rest of features we will impute the *nan* values with a replacement after analyzing the categorical data.

# Data transformation

### Categorical Data

In [ ]:
categorical_cols = ['lead_source', 'campaign_name', 'group_source', 'group_landing', 'type', 'role']
for cat in categorical_cols:
    df[cat] = df[cat].astype('category')
df.info()

Let´s analyze one by one by identifying the number of categories per feature and the counts we have for each category.

#### lead_source

In [ ]:
# number of unique categories
len(df.lead_source.unique())

In [ ]:
df.lead_source.value_counts()

#### campaign name

In [ ]:
len(df.campaign_name.unique())

In [ ]:
df.campaign_name.value_counts()

#### group_source

In [ ]:
len(df.group_source.unique())

In [ ]:
df.group_source.value_counts()

#### group_landing

In [ ]:
len(df.group_landing.unique())

In [ ]:
df.group_landing.value_counts()

#### type

In [ ]:
len(df.type.unique())

In [ ]:
df.type.value_counts()

#### role

In [ ]:
len(df.role.unique())

In [ ]:
df.role.value_counts()

According to these results, the fields that might require some transformations/actions are:

1. campaign_name with more than 300 different categories
2. role with more than 100 categories plus some weird values in other languages

In [ ]:
df.campaign_name.describe()

In [ ]:
# TODO this is not changing the range of the original category code
# TODO we would need to scale this feature too with MinMax Scaler
# we need to add the new category first
df.campaign_name = df.campaign_name.cat.add_categories(["other"])
df = df.apply(lambda x: x.mask(x.map(x.value_counts())<100, "other") if x.name == "campaign_name"  else x)
df.campaign_name.describe()

We managed to reduce from 317 to 33 categories. Now we repeat the same with the role feature.

In [ ]:
df.role.describe()

In [ ]:
# we are grouping minority roles within minor category
df.role = df.role.cat.add_categories(["minor"])
df = df.apply(lambda x: x.mask(x.map(x.value_counts())<100, "minor") if x.name == "role"  else x)
df.role.describe()

Successfully reduced from 120 categories (119 plus Nan) to 33 (32 plus Nan). The next step with categorical data would be to generate some identifier for each value and have a new feature for each selected field. However it is time to do something with those Nans in the dataset.

In [ ]:
# for type is going to be a new category "unknown"
df.type = df.type.cat.add_categories(["unknown"])
df.type = df.type.fillna("unknown")

In [ ]:
# for role is going to be the existing category "unknown"
df.role = df.role.fillna("unknown")

In [ ]:
# for place_within_tenant is going to be 0
df.place_within_tenant = df.place_within_tenant.fillna(0)

In [ ]:
# for continent, region_code, country_code and city the percentage is very low
# we decided to replace it with a 0, anyway it is a 5% so it is not going to affect much the distribution
for col in columns_fix:
    df[col] = df[col].fillna(0)

In [ ]:
# for nondst_utc_offset we decided to use an extreme value 27, again it is a 5%
df.nondst_utc_offset = df.nondst_utc_offset.fillna(27)

In [ ]:
# for campaign_name, group_source, group_landing, lead_source is going to be a new category "missing"
columns_miss = ["campaign_name", "group_source", "group_landing", "lead_source"]
for col in columns_miss:
    df[col] = df[col].cat.add_categories(["missing"])
    df[col] = df[col].fillna("missing")

We are good to go with the next step.

## Numerical data

Let´s paint some distributions of the numerical fields to check ranges furthermore and to identify where we might need to do some scaling.

In [ ]:
sns.set(style="darkgrid", palette="rocket")
sns.distplot(df.place_within_tenant)

In [ ]:
sns.distplot(df.continent)

In [ ]:
sns.distplot(df.country_code)

In [ ]:
sns.distplot(df.city)

In [ ]:
sns.distplot(df.region_code)

In [ ]:
sns.distplot(df.nondst_utc_offset)

### Scaling

The *continent* and *place_within_tenant* fields have only very few possible values, while *city* has a clear huge range of valid values. Besides the distribution of *place_within_tenant* is very skewed because of the outlier we already saw previously. Many machine learning algorithms perform better or converge faster when features are on a relatively similar scale and/or close to normally distributed. 
We are going to apply **MinMax Scaler** as it preserves the shape of the original distribution.

In [ ]:
columns_scale = ["place_within_tenant", "continent", "country_code", "city", "region_code", "nondst_utc_offset"]
for col in columns_scale:
    # the min max scaler requires a vector
    transformer = MinMaxScaler().fit(df[col].values.reshape(-1, 1)) # single feature
    transformed_data = transformer.transform(df[col].values.reshape(-1, 1))
    df[col+"_mm"] = transformed_data[:,0]

Let´s check the new range of the features.

In [ ]:
sns.distplot(df.place_within_tenant_mm)

In [ ]:
sns.distplot(df.continent_mm)

In [ ]:
sns.distplot(df.country_code_mm)

In [ ]:
sns.distplot(df.city_mm)

In [ ]:
sns.distplot(df.region_code_mm)

In [ ]:
sns.distplot(df.nondst_utc_offset_mm)

## Categorical columns into Numerical

In [ ]:
for col in df.columns:
    if df[col].dtype.name == 'category':
        df[col + "_cat"] = df[col].cat.codes

In [ ]:
df.head()

# Correlation analysis

In [ ]:
df.columns.get_loc("label")

Selecting all transformed new columns after the *label* column:

In [ ]:
sel_df = df.iloc[:,12:]
sel_df.head()

In [ ]:
sel_df.info()

In [ ]:
plt.rcParams["figure.figsize"] = (20,10)
plt.tight_layout()

def paint_correlation_matrix(data):
    #Draw correlation mtx
    k = data.count(axis=1)[0] 
    corrmat = data.corr()
    cols = corrmat.nlargest(k, 'label')['label'].index
    cm = np.corrcoef(data[cols].values.T)
    sns.set(font_scale=1.25)
    hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
    plt.show()
    
paint_correlation_matrix(sel_df)

There is not a strong correlation with the output we want to predict, given the provided data. The only field having the "highest" value is *lead_source*. Besides there is a strong correlation between the *utc_offset* field and the region, continent, city and country code. Something that is not a surprise given that the timezone is totally dependent on the geographical place, and all those fields are connected with the region. Then we could delete the utc_offset field from relevant predictors.

We can identify also a strong correlation between the geographical fields (country, continent, city and region), again expected. Thus we could keep city and region_code, since they both have the lowest correlation, and discard continent and country_code.

In [ ]:
sel_df.drop(['nondst_utc_offset_mm', 'continent_mm', 'country_code_mm'], axis=1, inplace=True)

## Extra test: Using the non-scaled features

Including the same correlation matrix but using the non-scaled features just to probe that correlation factors were not affected.

In [ ]:
raw_df = df.drop(["place_within_tenant_mm", "continent_mm", "country_code_mm", "city_mm", "region_code_mm", "nondst_utc_offset_mm"], axis=1)

In [ ]:
paint_correlation_matrix(raw_df)

# Saving final dataset

In [ ]:
sel_df.to_pickle("training_df")

## References

https://realpython.com/python-pandas-tricks/#5-use-categorical-data-to-save-on-time-and-space

https://jakevdp.github.io/PythonDataScienceHandbook/03.04-missing-values.html

https://machinelearningmastery.com/how-to-score-probability-predictions-in-python/

https://towardsdatascience.com/scale-standardize-or-normalize-with-scikit-learn-6ccc7d176a02?gi=d46d94ecce1

https://scikit-learn.org/stable/modules/preprocessing.html